## **Exercise**
1. Use the diabetes.csv dataset to do the following:​

Select the following 4 attributes (3 features + 1 class label) :​

Glucose, BloodPressure, Insulin, Outcome​

2. Normalize Glucose, BloodPressure and Insulin to [0, 1] using MinMax.​

3. Store the new data (3 normalized features + 1 class label) in another dataset S.​

4. Modify the MQTT example to do the following:​

The publisher publishes records in S continuously. When it reaches the end of S, it continues to send from the beginning again.​

The subscriber continuously receives the data. For each latest record r received, apply the 3NN classification to the last 5 records before r, and compare the classification result with the Outcome label in r.​

Repeat this for 1000 times, and report the number of correct classifications. ​

​


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time
import paho.mqtt.client as mqtt

In [2]:
df_diabetes = pd.read_csv("diabetes.csv")

### 1. Use the diabetes.csv dataset to do the following:​

Select the following 4 attributes (3 features + 1 class label) :​

Glucose, BloodPressure, Insulin, Outcome​

In [3]:
df_S = df_diabetes[['Glucose', 'BloodPressure', 'Insulin', 'Outcome']].copy()

In [4]:
df_S.head()

,Glucose,BloodPressure,Insulin,Outcome
0,148,72,0,1
1,85,66,0,0
2,183,64,0,1
3,89,66,94,0
4,137,40,168,1


In [5]:
df_S.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Glucose        768 non-null    int64
 1   BloodPressure  768 non-null    int64
 2   Insulin        768 non-null    int64
 3   Outcome        768 non-null    int64
dtypes: int64(4)
memory usage: 24.1 KB


2. Normalize Glucose, BloodPressure and Insulin to [0, 1] using MinMax.​

In [6]:
min_max_scaler = MinMaxScaler()
df_S[['Glucose', 'BloodPressure', 'Insulin']] = min_max_scaler.fit_transform(df_S[['Glucose', 'BloodPressure', 'Insulin']])

In [ ]:
print(df_S.head())

    Glucose  BloodPressure   Insulin  Outcome
0  0.743719       0.590164  0.000000        1
1  0.427136       0.540984  0.000000        0
2  0.919598       0.524590  0.000000        1
3  0.447236       0.540984  0.111111        0
4  0.688442       0.327869  0.198582        1


: 

loaded the diabetes dataset, selected the 4 desired columns, and copyed them into a new DataFrame S.
used MinMaxScaler to normalize Glucose, BloodPressure, and Insulin to the [0, 1] range.

In [ ]:
records = df_S.to_dict('records')
period = len(records)

# --- Set Up MQTT Publisher ---
mqttc = mqtt.Client()
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

if __name__ == "__main__":
    print("Publisher: Starting data publication...")
    index = 0
    while True:
        # Cycle through records continuously
        record = records[index % period]
        # Publish the record as a JSON string (you could use json.dumps if needed)
        mqttc.publish("myroom/diabetes", str(record))
        # Publish a new record every 2 seconds
        time.sleep(0.1)
        index += 1

Publisher: Starting data publication...
